In [ ]:
#Import sqlite
import sqlite3

In [ ]:
#import DB
db=sqlite3.connect("herewegoDB.db")

In [ ]:
#create cursor
cursor = db.cursor()

In [ ]:
#to test the connection
cursor.execute('''select * FRom users''')
p= cursor.fetchall()
for j in p:
    print(p)


In [ ]:
#determines the role of a user provided user is in the database
def get_user_role(username):
	role = ""
	cursor.execute('''
		SELECT user_role FROM users
		WHERE username = ?
		''',
		(username,)
		)
	role = cursor.fetchone()
	if role is None:
		return "User '%s' is not found or user does not have a role" %username
	else:
		return role[0]

In [ ]:
get_user_role("aditya")

In [ ]:
def change_event_capacity(event_name,capacity):
	cursor.execute('''
			UPDATE events SET event_capacity = ?
			WHERE event_name = ?
		''', (capacity,event_name))
	db.commit()
	cursor.execute('''
					SELECT event_name, event_capacity 
					FROM events WHERE event_name=?''', (event_name,))
	event_detail = cursor.fetchone()
	if event_detail is None:
		return "Event name '%s' has not been created. Please ensure the right event has been entered or created and try again" %event_name
	else:
		return event_detail

In [ ]:
change_event_capacity("singing",1)

In [ ]:
import datetime

In [ ]:
def add_user(user_account,user_id,first_name,last_name,age,email,user_role,user_phone,user_name):
	try:		
		creation_date = datetime.datetime.now().strftime("%Y-%m-%d")
		print(get_user_role(user_account))
		if get_user_role(user_account) != 'admin':
			return "User %s doesn't exist or does not have appropriate permission to add other users" %user_account
		else:
			cursor.execute('''
							INSERT INTO users
							VALUES(?,?,?,?,?,?,?,?,?)
				''',(user_id,first_name,last_name,age,email,user_role,user_phone,creation_date,user_name)
				)
			db.commit()
			cursor.execute('''
							SELECT * FROM users
				''')
			users = cursor.fetchall()
			for user in users:
				print(user)
	except sqlite3.Error as e:
		print("Error {" + e.args[0] + "}") 

In [ ]:
add_user("darasimi",5,"jon","snow",25,"jon.snow@got.com","user","21597648","stevendara")

In [ ]:
import datetime

In [ ]:
def fetch_venue(venue_name):
    cursor.execute("SELECT * FROM venues WHERE venue_name=?",(venue_name,))
    all= cursor.fetchall()
    return(all[0])
        

In [ ]:
fetch_venue('rainey')

In [ ]:
#add a venue, 
#only admin can add or update venues#added by aditya

def add_venue(user_account,venue_name,venue_open,venue_close,zip_code,city,address):
    user_role = get_user_role(user_account)
    try:
        if user_role!='admin':
            return "You do not have rights to create or update venues"
        else:            
            cursor.execute('''
            insert into venues(venue_name,venue_open,venue_close,zip_code,city,address)
            values(?,?,?,?,?,?)
            ''',(venue_name,venue_open,venue_close,zip_code,city,address))
            db.commit()
            #cursor.execute("SELECT * FROM venues WHERE venue_name=?",(venue_name,))
            all= cursor.fetchall()
            for p in all:
                print(p)
            
    except sqlite3.Error as e:
         print("Error {" + e.args[0] + "}")
        

In [ ]:
add_venue('aditya',"Rutwick's House",'8:00','20:00',"78751","Austin","3115 Helms Street")

In [ ]:
#Display timeslot availability at a venue#added by aditya
#This function accepts venue name as an input
def venue_avail(venue_name):
    cursor.execute('''SELECT v.venue_name,vc.slot_start,vc.slot_end,vc.is_available,v.zip_code,v.city,v.address FROM venue_calendar vc,venues v WHERE vc.is_available = 'Y' and vc.venue_id = v.venue_id and v.venue_name=?''',(venue_name,))
    all= cursor.fetchall()
    for p in all:
        print(p)  

In [ ]:
venue_avail('rainey')

In [ ]:
import datetime

In [ ]:
#List events at a venue by date/time
def display_venue_events(venue_name,event_date,event_start):
    try:
        cursor.execute('''select v.venue_name,e.event_name,e.event_date,e.event_start,e.event_end from events e, venues v where e.venue_id = v.venue_id 
        and v.venue_name = ? and event_date = ? and event_start = ?
        ''',(venue_name,event_date,event_start,))
        all = cursor.fetchall()
        for p in all:
            print(p)
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")
            



In [ ]:
display_venue_events("rainey","2019-07-26","10:00")

In [ ]:
#check current event capacity # added by aditya
#it accepts event_id as the input and returns the current event capacity
def check_capacity(event_id):
    try:
        cursor.execute('''select event_capacity FRom events
                       where event_id = ?
                       ''',(event_id,))
        current_cap = cursor.fetchone()
        return (current_cap[0])
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")

In [ ]:
check_capacity(2)

In [ ]:
# Function implemented to join an event.
# added by aditya. It accepts the user id and event id as input
def join_event(username,event_id):
    msg = "sucess"    
    try:
        event_cap = check_capacity(event_id)
        print (event_cap)
        if event_cap >0:
            #print Make entry into user events table and reduce the flag count
            cursor.execute('''select user_id from users where username=?
                        ''',(username,))
            user_result = cursor.fetchone()
            user_id = user_result[0]
            #make entries into the user events table
            cursor.execute('''insert into user_events(user_id,event_id)
                              values(?,?)''',(user_id,event_id,))
            #increase the event capacity count            
            cursor.execute('''update events set event_capacity=event_capacity-1 
                           where event_id = ?
                           ''',(event_id,))
            db.commit()
            return msg
            
        else:
            print ("Please join later")
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")
        

In [ ]:
join_event("stevendara",2)

In [ ]:
#remove an event. Added by Aditya # this feature only lets the admin remove an event.
# it accepts the event id and username as input.
# Steps:
# 1. Delete the entries from user_events table
# 2. Delete the entries from the events table
# 3. Free up the venues calendar ---****yet to be implemented**

def remove_event(event_id,username):
    msg = "Event Succesfully Deleted"
    try:
              
        if get_user_role(username)=="admin":
            cursor.execute('''delete from user_events where event_id =?
                           ''',(event_id,))
            cursor.execute('''delete from events where event_id =?
                           ''',(event_id,))
            db.commit()
            return msg
        else:
            print("You are not authorized to perform this action")
    
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")





In [ ]:
remove_event(1,"")

In [ ]:
#User opts out of an event. Added by Aditya # this feature allows the users to exit an event.
#1. This should remove the user entry from the user_events table for a particular event
#2. Event capacity should be increased by one everytime a user quits an event.
def quit_event(username,event_id):
    msg = "User removed from the event succesfully"
    
    try:
        cursor.execute('''select user_id From users where username = ?
                      ''',(username,))
        user_result = cursor.fetchone()
        user_id = user_result[0]
        
        
        cursor.execute('''delete from user_events where user_id =?
                       ''',(user_id,))
        
        cursor.execute('''update events set event_capacity=event_capacity+1
                       where event_id =?
                       ''',(event_id,))
        db.commit()
        return msg
        
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")

In [ ]:
quit_event("stevendara",2)

In [ ]:
#Display timeslot availability at a venue#added by aditya
#This function accepts venue name as an input
def venue_avail(venue_name):
    cursor.execute('''SELECT v.venue_name,vc.slot_start,vc.slot_end,vc.is_available,v.zip_code,v.city,v.address FROM venue_calendar vc,venues v WHERE vc.is_available = 'Y' and vc.venue_id = v.venue_id and v.venue_name=?''',(venue_name,))
    all= cursor.fetchall()
    for p in all:
        print(p) 

In [ ]:
#Returns boolean value if the venue is available, else returns false
#lot availability at a venue#added by aditya
#This function accepts venue name as an input
def venue_avail(venue_name,venue_date,start_time):
    try:
        print("holy cow")        
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")

In [ ]:
venue_avail("rainjesaasay")

In [ ]:
#Start an event. ANy user or admin can start an event.
#This function checks if the time slot requested by the user is available
#If the slot is available it books the time slot and makes the time slot unavailable for booking

def venue_available():
    try:
        
        cursor.execute('''SELECT v.venue_name,vc.slot_start,vc.slot_end,vc.is_available,v.zip_code,v.city,v.address
        FROM venue_calendar vc,venues v WHERE vc.is_available = 'Y' and vc.venue_id = v.venue_id and
        v.venue_name='uptown austin' and slot_start ="12:05"
        ''')
        result = cursor.fetchone()
        #print(result[0])
        if  result is None:
            return False
        else:
            return True
            
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")
        


In [ ]:
venue_available()

In [ ]:
db.close()